# End-to-End Demonstration: Schoology

*Goal*: run the Schoology Extractor and then upload the results into an `LMS` database.

Execution time is highly dependent on the number of students and the number of sections in your installation of Schoology.

## Software Requirements

1. Be sure to install Python 3.9; if you have multiple versions, make sure that the `python` command runs version 3.9.x. You can confirm your version by running `python --version` at a command prompt.
1. Microsoft SQL Server 2017 or 2019, in Windows or Linux.

## Getting Started

1. Confirm you have [poetry](https://python-poetry.org) installed (`poetry --version`).
1. Follow the [notebook instructions](README.md) to install dependencies used by this notebook.
1. Create an `LMS` database in SQL Server.
1. Please read through the notebook and fill in the two text boxes below before executing the entire notebook.

In [ ]:
# Load some utilities
from IPython.display import display, Markdown
import ipywidgets as widgets

# Setup logging
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

## Prepare Input Data

Update the variables in the next block as needed.

In [ ]:
LOG_LEVEL = "INFO"
OUTPUT_DIRECTORY = "s-data"
SYNC_DATABASE_DIRECTORY=OUTPUT_DIRECTORY
DB_ENGINE = "mssql"
DB_SERVER = "localhost"
DB_NAME = "LMS"
DB_PORT = 1433
EXTRACT_ASSIGNMENTS = True
EXTRACT_ACTIVITIES = False
EXTRACT_ATTENDANCE = False
EXTRACT_GRADES = False

Run the next cell to create text input boxes for providing the agency's Schoology API key and secret. You can find your key and secret at [https://app.schoology.com/api](https://app.schoology.com/api). Then run the next cell down to capture the values before running the last cells.

In [ ]:
def get_key_secret(key, secret):
    return key, secret

key_widget = widgets.Text(
    description="Key"
)
secret_widget = widgets.Text(
    description="Secret"
)
result = widgets.interactive(get_key_secret, key=key_widget, secret=secret_widget)
display(result)

In [ ]:
KEY = result.children[0].value
SECRET = result.children[1].value
print("Captured token and base URL.")

## Run the Schoology Extractor

In [ ]:
from edfi_schoology_extractor.helpers.arg_parser import MainArguments as s_args
from edfi_schoology_extractor import extract_facade

arguments = s_args(
    client_key=KEY,
    client_secret=SECRET,
    output_directory=OUTPUT_DIRECTORY,
    log_level=LOG_LEVEL,
    page_size=200,
    input_directory=None,
    sync_database_directory=SYNC_DATABASE_DIRECTORY,
    extract_assignments=EXTRACT_ASSIGNMENTS,
    extract_activities=EXTRACT_ACTIVITIES,
    extract_attendance=EXTRACT_ATTENDANCE,
    extract_grades=EXTRACT_GRADES,
)

extract_facade.run(arguments)

## Run the Learning Management System Data Store Loader (LMS-DS-Loader)

The default setup below uses Windows integrated security. For username/password security, please review the commented-out code.

In [ ]:
from edfi_lms_ds_loader.helpers.argparser import MainArguments as lms_args
from edfi_lms_ds_loader import loader_facade

arguments = lms_args(
    OUTPUT_DIRECTORY,
    DB_ENGINE,
    LOG_LEVEL
)
arguments.set_connection_string_using_integrated_security(
    DB_SERVER,
    DB_PORT,
    DB_NAME,
)
# For password auth, comment out the line above and uncomment this one:
# arguments.set_connection_string(
#     DB_SERVER,
#     DB_PORT,
#     DB_NAME,
#     USERNAME,
#     PASSWORD,
# )

loader_facade.run_loader(arguments)